<a href="https://colab.research.google.com/github/maxreynolds/Pattern-Cloud/blob/main/accelerate_via_subprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cd /content/
!git clone https://github.com/maxreynolds/diffusers.git
!pip install ./diffusers
!pip install -U -r /content/diffusers/examples/text_to_image/requirements.txt

In [ ]:
!pip install -q wandb

In [ ]:
import wandb
wandb.login()

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
!accelerate config default --mixed_precision fp16

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [24]:
%%writefile test.py
import subprocess
import numpy as np

subprocess.run(["accelerate", "launch", "diffusers/examples/text_to_image/train_text_to_image.py",
  "--pretrained_model_name_or_path=CompVis/stable-diffusion-v1-4",
  "--dataset_name=MaxReynolds/Lee_Souder_Combined",
  "--use_ema",
  "--resolution=512",
  "--center_crop",
  "--random_flip",
  "--train_batch_size=1",
  "--gradient_accumulation_steps=4",
  "--gradient_checkpointing",
  "--max_train_steps=400",
  "--learning_rate=1e-05",
  "--max_grad_norm=1",
  "--checkpointing_steps=100000",
  "--lr_scheduler=constant",
  "--lr_warmup_steps=0",
  "--push_to_hub",
  "--output_dir=MaxReynolds/MyStreamlitModel",
  "--validation_prompt=Rocket Launcher by Lee Souder",
  "--report_to=wandb"])



Overwriting test.py


In [25]:
!python test.py

2023-10-30 20:47:36.682335: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-30 20:47:36.682395: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-30 20:47:36.682441: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-30 20:47:37.912546: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
10/30/2023 20:47:38 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

{'dynamic_thresholding_ratio', 'clip_sample_range', 'timestep_spacing',

In [21]:
!ls


diffusers  sample_data	sd-model-finetuned  test.py
